In [14]:
import numpy as np
import pandas as pd
import collections
import itertools
import ast
import random

Importar Trabajo previo

In [8]:
resumen = pd.read_csv('preprocesado.csv', converters={"shingles": ast.literal_eval})

Consolidar FHs a partir de los archivos guardados en disco.

In [9]:
FH = np.genfromtxt('output/file0.csv', delimiter=',')
for i in range(1, 10):
    FH2 = np.genfromtxt(f'output/file{i}.csv', delimiter=',')
    FH = np.concatenate((FH, FH2), axis=1)
FH.shape

(20, 203726)

## Banding

Mostrar distancia mínima a la que debe estar dos tweet para ser considerados pares candidatos

In [10]:
r = 5
b = 4
(1/b)**(1/r)

0.757858283255199

In [11]:
# https://towardsdatascience.com/locality-sensitive-hashing-how-to-find-similar-items-in-a-large-set-with-precision-d907c52b05fc
sig_mat = FH
n, d = sig_mat.shape
#assert(n==b*r)
hashbuckets = collections.defaultdict(set)
bands = np.array_split(sig_mat, b, axis=0)
for i,band in enumerate(bands):
    for j in range(d):
        # The last value must be made a string, to prevent accidental
        # key collisions of r+1 integers when we really only want
        # keys of r integers plus a band index
        band_id = tuple(list(band[:,j])+[str(i)])
        hashbuckets[band_id].add(j)
candidate_pairs = set()
for bucket in hashbuckets.values():
    if len(bucket) > 1:
        for pair in itertools.combinations(bucket, 2):
            candidate_pairs.add(pair)
print(f"Obtuvimos {len(candidate_pairs)} pares candidatos, con b = {b} y r = {r}")

Obtuvimos 2235158 pares candidatos, con b = 4 y r = 5


Obtener pares candidatos

## Limpiar pares de candidatos

Quitar tweets idénticos de los pares similares

In [12]:
trimmed_cadidate_pairs = []
for pair in candidate_pairs:
        a, b = pair
        text_a = resumen.text[a]
        text_b = resumen.text[b]
        if (text_a != text_b) and ('http' not in text_a) and ('http' not in text_b):
                trimmed_cadidate_pairs.append((a, b))
print(f"Quedan {len(trimmed_cadidate_pairs)} pares candidatos")

Quedan 1035758 pares candidatos


Mostrar 3 pares de tweets presuntamente similares

In [15]:
for pair in random.sample(list(trimmed_cadidate_pairs), 3):
    a, b = pair
    print(f"1. {resumen.screen_name[a]}: ",resumen.text[a])
    print(f"2. {resumen.screen_name[b]}: ",resumen.text[b])
    print("----------")

1. GallinatoRojas:  @daniponcev @AlejoLatigo @rocicantuarias @tere_marinovic Ibas tan bien. Hasta que ofendes. 🤦
2. vfvmdechile:  @cat_botas @AlejandroZagal4 @JorgeVe84124650 @AlejoLatigo @rocicantuarias @tere_marinovic Chao gato fleto disfruta.
----------
1. Isitacake:  @patriciapolitz @convencioncl @sebastian_gray @Censelio @rkatrileo @TiareHey Que tiene que ver una cosa con otra... Por favor!!
2. mptellechea:  @patriciapolitz @convencioncl @sebastian_gray @Censelio @rkatrileo @TiareHey Tan idiota esta judía nazi
----------
1. Eva59523400:  @patriciapolitz @iSci @uchile @gabrielboric @LeoBassoSotz Karmaaaa, hace lo tuyo con esta mujer, jajajaja.
2. maxberrios24:  @pennaleon @la_toletole @patriciapolitz @iSci @uchile @gabrielboric @LeoBassoSotz La Danza ...Y es "profesora" PLOP
----------


## Identificar autores similares

Contar cuantos tweets similares hay para cada par de autores considerados candidatos similares

In [16]:
count_dict = dict()
for pair in trimmed_cadidate_pairs:
    a, b = pair
    user_a = resumen.screen_name[a]
    user_b = resumen.screen_name[b]
    if user_a != user_b:
        try:
            count_dict[user_a][user_b] += 1
        except KeyError:
            try:
                count_dict[user_a] = dict()
                count_dict[user_a][user_b] += 1
            except KeyError:
                count_dict[user_a][user_b] = 1
        try:
            count_dict[user_b][user_a] += 1
        except KeyError:
            try:
                count_dict[user_b] = dict()
                count_dict[user_b][user_a] += 1
            except KeyError:
                count_dict[user_b][user_a] = 1

Mostrar solo los autores que están por sobre un threshold de tweets similares

In [17]:
count_df = pd.DataFrame(count_dict).reset_index()
count_long = pd.melt(count_df, id_vars='index')
similar_authors = count_long[count_long.value >= 3].rename({'index': 'user_1', 'variable': 'user_2'}, axis = 1)
similar_authors = similar_authors[similar_authors.value <= 30]
similar_authors

,user_1,user_2,value
2153843,salvador_3_68,enalvareli,3.0
2890499,salvador_3_68,pamelasexi18,18.0
2893171,pamelasexi18,salvador_3_68,5.0
3605273,Quenna4,Palomaenlibert2,5.0
5851797,RomilioValenzu4,soydelever69,3.0
8418034,Tol1Soto,marcelo_arrieta,12.0
9948202,Tol1Soto,0_sonero,5.0
10367328,A_Erazo_P,bmachuca2,3.0
10739388,mono_dex,e_constituyente,4.0


Función para imprimir tweets de dos candidatos pares:

In [18]:
def show_tweets_from_candidates(user_1, user_2):
    for pair in trimmed_cadidate_pairs:
        a, b = pair
        user_a = resumen.screen_name[a]
        user_b = resumen.screen_name[b]
        if ((user_a == user_1) and (user_b == user_2)) or ((user_b == user_1) and (user_a == user_2)):
            print("By user ", user_a,":\n",  f'"{resumen.text[a]}"\n')
            print("By user ", user_b, ":\n", f'"{resumen.text[b]}"')
            print("_______________________________________________________\n")

Mostrar pares de tweets de todos los autores similares

In [19]:
for index, row in similar_authors.iterrows():
    user_1 = row.user_1
    user_2 = row.user_2
    show_tweets_from_candidates(user_1, user_2)
    

By user  enalvareli :
 "#CongresoBasuraYGolpista 
#CierrenElCongreso 
#CierrenElCongreso #NuevaConstitucion #grupoelcomercio #Gestion"

By user  salvador_3_68 :
 "#CierreDelCongreso  #NuevaConstitucion 
#PrensaBasuraYGolpista 
Castillo Keiko Fujimori
Xi Jinping FIFA Metro"
_______________________________________________________

By user  enalvareli :
 "#CongresoBasuraYGolpista 
#CierrenElCongreso 
#CierrenElCongreso #NuevaConstitucion"

By user  salvador_3_68 :
 "#CierreDelCongreso  #NuevaConstitucion 
#PrensaBasuraYGolpista 
Sigrid María del Carmen Alva
Yotun Guerrero"
_______________________________________________________

By user  enalvareli :
 "#CongresoBasuraYGolpista 
#CierrenElCongreso 
#CierrenElCongreso #NuevaConstitucion"

By user  salvador_3_68 :
 "#CierrenElCongreso  #NuevaConstitucion 
#PrensaBasuraYGolpista 
Willax limaflores Bruno Pacheco"
_______________________________________________________

By user  salvador_3_68 :
 "#CierreDelCongreso  #NuevaConstitucion 
#PrensaB

KeyboardInterrupt: 